# Data Centre modelling
## Ryan Jenkinson

In [1]:
import pypsa
import pandas as pd
import numpy as np

In [3]:
# Perform consistency check
try:
    n.consistency_check()
    print("Network consistency check passed before solving.")
except UserWarning as e:
    print(f"Consistency check warning before solving: {e}")
except Exception as e:
    print(f"Consistency check error before solving: {e}")
    # Depending on the error, you might want to exit or handle it
    # exit()


	gas_turbine
Index(['electricity_bus', 'bess_connection_bus'], dtype='object', name='Bus')
Index(['bess_charger', 'bess_discharger'], dtype='object', name='Link')
Index(['bess_storage'], dtype='object', name='Store')


Network consistency check passed before solving.


In [4]:
import pandas as pd
import numpy as np
import pypsa

# Define simulation period: a full year, hourly
snapshots = pd.date_range(start="2023-01-01 00:00", end="2023-12-31 23:00", freq="h")
n = pypsa.Network(snapshots=snapshots)

# --- DEFINE CARRIERS ---
# It's good practice to define all carriers upfront.
n.add("Carrier", "AC") # For AC buses
n.add("Carrier", "gas", co2_emissions=0.185) # tCO2/MWh_th, typical for natural gas
n.add("Carrier", "offshore_wind")
n.add("Carrier", "battery") # Carrier for battery components

# --- DEFINE BUSES ---
# Bus: A single electrical bus will represent the microgrid's point of common coupling.
n.add("Bus", "electricity_bus", carrier="AC")
# Add a dedicated bus for the BESS connection point
n.add("Bus", "bess_connection_bus", carrier="AC")


# --- DEFINE LOAD ---
# Load (Data Centre): The data centre is modelled as a load with a constant power demand.
dc_power_mw = 115  # MW, constant demand including PUE
# Create an hourly demand profile as a pandas Series
hourly_demand_profile = pd.Series(dc_power_mw, index=n.snapshots)
n.add("Load", "data_centre_load", bus="electricity_bus", p_set=hourly_demand_profile)


# --- DEFINE GENERATORS ---

# Generator (Offshore Wind Farm): The offshore wind farm's capacity is extendable.
np.random.seed(42) # for reproducibility
wind_availability_pu = pd.Series(np.random.rand(len(n.snapshots)) * 0.6 + 0.2, index=n.snapshots)
wind_availability_pu = (wind_availability_pu / wind_availability_pu.mean()) * 0.55 # Ensure average matches target CF
wind_availability_pu = wind_availability_pu.clip(0, 1) # Ensure values are between 0 and 1

wind_capex_per_mw = 2370000  # £ per MW
wind_lifetime_years = 27
wind_discount_rate = 0.07
annuitised_wind_cost = (wind_capex_per_mw * wind_discount_rate) / (1 - (1 + wind_discount_rate)**-wind_lifetime_years)
wind_fixed_opex = 76000  # £ per MW per year (fixed O&M)
wind_capital_cost = annuitised_wind_cost + wind_fixed_opex  # £ per MW per year
wind_marginal_cost = 0.01  # £ per MWh (Variable O&M, assumed small)

n.add("Generator", "offshore_wind_farm",
      bus="electricity_bus",
      carrier="offshore_wind",
      p_nom_extendable=True,
      capital_cost=wind_capital_cost,
      marginal_cost=wind_marginal_cost,
      p_max_pu=wind_availability_pu)


# Generator (Gas Turbine Backup): The gas turbine provides backup, and its capacity is also extendable.
gas_capex_per_mw = 631000  # £ per MW
gas_lifetime_years = 25
gas_discount_rate = 0.07
annuitised_gas_cost = (gas_capex_per_mw * gas_discount_rate) / (1 - (1 + gas_discount_rate)**-gas_lifetime_years)
gas_fixed_opex = 12600  # £ per MW per year (fixed O&M)
gas_capital_cost = annuitised_gas_cost + gas_fixed_opex  # £ per MW per year

gas_fuel_price_mwh = 60  # £ per MWh_fuel (assumed fuel price)
gas_efficiency = 0.34  # Efficiency of gas turbine (MWh_elec / MWh_fuel)
gas_variable_operation_and_management_mwh = 7.05  # £ per MWh_elec (non-fuel variable O&M)
gas_marginal_cost_fuel = gas_fuel_price_mwh / gas_efficiency  # £ per MWh_elec
gas_marginal_cost = gas_marginal_cost_fuel + gas_variable_operation_and_management_mwh  # Total marginal cost

n.add("Generator", "gas_turbine",
      bus="electricity_bus",
      carrier="gas",
      p_nom_extendable=True,
      committable=False, # *** CHANGED: Cannot be both committable and extendable for LOPF capacity optimization ***
      capital_cost=gas_capital_cost,
      marginal_cost=gas_marginal_cost)


# --- DEFINE STORAGE (BESS) ---

# Storage (BESS - for sensitivity analysis): Battery storage parameters
bess_energy_capex_per_mwh = 225000  # £ per MWh
bess_power_capex_per_mw = 130000  # £ per MW
bess_lifetime_years = 15
bess_discount_rate = 0.07

annuitised_bess_energy_cost = (bess_energy_capex_per_mwh * bess_discount_rate) / (1 - (1 + bess_discount_rate)**-bess_lifetime_years)
bess_capital_cost_energy = annuitised_bess_energy_cost # Cost for energy capacity (Store)

annuitised_bess_power_cost = (bess_power_capex_per_mw * bess_discount_rate) / (1 - (1 + bess_discount_rate)**-bess_lifetime_years)
bess_fixed_opex_per_mw_power = 20000 # Fixed OPEX for power conversion part
bess_capital_cost_power = annuitised_bess_power_cost + bess_fixed_opex_per_mw_power # Cost for power capacity (Links)

bess_round_trip_efficiency = 0.87
standing_loss_per_hour = 0.001
bess_marginal_cost_links = 0.001 # Marginal cost for charging/discharging

# Store component representing the energy capacity of the BESS
n.add("Store", "bess_storage",
      bus="bess_connection_bus",
      carrier="battery", # This carrier should now be recognized
      e_nom_extendable=True,
      e_cyclic=True,
      standing_loss=standing_loss_per_hour,
      capital_cost=bess_capital_cost_energy)

# Link component for charging the BESS
n.add("Link", "bess_charger",
      bus0="electricity_bus",
      bus1="bess_connection_bus",
      carrier="battery", # This carrier should now be recognized
      p_nom_extendable=True,
      capital_cost=bess_capital_cost_power,
      efficiency=bess_round_trip_efficiency**0.5,
      marginal_cost=bess_marginal_cost_links)

# Link component for discharging the BESS
n.add("Link", "bess_discharger",
      bus0="bess_connection_bus",
      bus1="electricity_bus",
      carrier="battery", # This carrier should now be recognized
      p_nom_extendable=True,
      capital_cost=0, # Assuming power CAPEX is fully on charger link, or split if preferred
      efficiency=bess_round_trip_efficiency**0.5,
      marginal_cost=bess_marginal_cost_links)

# --- CONSISTENCY CHECK & SOLVE ---

# Perform consistency check
try:
    n.consistency_check()
    print("Network consistency check passed before solving.")
except UserWarning as e:
    print(f"Consistency check warning before solving: {e}") # Should be cleaner now
except Exception as e:
    print(f"Consistency check error before solving: {e}")
    # exit() # Optionally exit if errors are critical

# Ensure you have a solver installed (e.g., cbc, glpk, gurobi)
solver_name = 'glpk' # or 'glpk', 'gurobi', etc.
print(f"\nAttempting to solve the LOPF with solver: {solver_name}")

try:
    # Solve the linear optimal power flow problem
    n.lopf(solver_name=solver_name, 
           solver_options={'threads': 4} if solver_name == 'cbc' else {}, 
           pyomo=False, # Use linopy by default
           solver_logfile="solver.log"
           )
    print("LOPF solved successfully.")

    # --- DISPLAY RESULTS ---
    print(f"\n--- Optimization Results ---")
    # Check if objective value exists (solver might have failed)
    if hasattr(n, 'objective'):
        print(f"Total System Cost (Objective Value): £{n.objective:,.2f} per year")
    else:
        print("Objective value not found. Solver may have failed or problem might be infeasible/unbounded.")

    print("\nOptimal Capacities:")
    # Check if components exist in results before trying to access them
    if 'offshore_wind_farm' in n.generators.index:
        print(f"  Offshore Wind Farm (p_nom_opt): {n.generators.at['offshore_wind_farm', 'p_nom_opt']:.2f} MW")
    if 'gas_turbine' in n.generators.index:
        print(f"  Gas Turbine (p_nom_opt): {n.generators.at['gas_turbine', 'p_nom_opt']:.2f} MW")
    if 'bess_storage' in n.stores.index:
        print(f"  BESS Storage Energy Capacity (e_nom_opt): {n.stores.at['bess_storage', 'e_nom_opt']:.2f} MWh")
    if 'bess_charger' in n.links.index:
        print(f"  BESS Charger Power Capacity (p_nom_opt): {n.links.at['bess_charger', 'p_nom_opt']:.2f} MW")
    if 'bess_discharger' in n.links.index:
        print(f"  BESS Discharger Power Capacity (p_nom_opt): {n.links.at['bess_discharger', 'p_nom_opt']:.2f} MW")

except Exception as e:
    print(f"An error occurred during LOPF solution or results processing: {e}")
    print("Please ensure you have a compatible solver installed and in your PATH.")
    print("Common solvers: cbc, glpk. You might need to install them.")
    print("Also ensure Pyomo or Linopy (PyPSA's modeling backend) is installed.")



Network consistency check passed before solving.

Attempting to solve the LOPF with solver: glpk
An error occurred during LOPF solution or results processing: 'Network' object has no attribute 'lopf'
Please ensure you have a compatible solver installed and in your PATH.
Common solvers: cbc, glpk. You might need to install them.
Also ensure Pyomo or Linopy (PyPSA's modeling backend) is installed.


In [5]:
import pandas as pd
import numpy as np
import pypsa

# Define simulation period: a full year, hourly
snapshots = pd.date_range(start="2023-01-01 00:00", end="2023-12-31 23:00", freq="h")
n = pypsa.Network(snapshots=snapshots)

# --- DEFINE CARRIERS ---
# It's good practice to define all carriers upfront.
n.add("Carrier", "AC") # For AC buses
n.add("Carrier", "gas", co2_emissions=0.185) # tCO2/MWh_th, typical for natural gas
n.add("Carrier", "offshore_wind")
n.add("Carrier", "battery") # Carrier for battery components

# --- DEFINE BUSES ---
# Bus: A single electrical bus will represent the microgrid's point of common coupling.
n.add("Bus", "electricity_bus", carrier="AC")
# Add a dedicated bus for the BESS connection point
n.add("Bus", "bess_connection_bus", carrier="AC")


# --- DEFINE LOAD ---
# Load (Data Centre): The data centre is modelled as a load with a constant power demand.
dc_power_mw = 115  # MW, constant demand including PUE
# Create an hourly demand profile as a pandas Series
hourly_demand_profile = pd.Series(dc_power_mw, index=n.snapshots)
n.add("Load", "data_centre_load", bus="electricity_bus", p_set=hourly_demand_profile)


# --- DEFINE GENERATORS ---

# Generator (Offshore Wind Farm): The offshore wind farm's capacity is extendable.
np.random.seed(42) # for reproducibility
wind_availability_pu = pd.Series(np.random.rand(len(n.snapshots)) * 0.6 + 0.2, index=n.snapshots)
wind_availability_pu = (wind_availability_pu / wind_availability_pu.mean()) * 0.55 # Ensure average matches target CF
wind_availability_pu = wind_availability_pu.clip(0, 1) # Ensure values are between 0 and 1

wind_capex_per_mw = 2370000  # £ per MW
wind_lifetime_years = 27
wind_discount_rate = 0.07
annuitised_wind_cost = (wind_capex_per_mw * wind_discount_rate) / (1 - (1 + wind_discount_rate)**-wind_lifetime_years)
wind_fixed_opex = 76000  # £ per MW per year (fixed O&M)
wind_capital_cost = annuitised_wind_cost + wind_fixed_opex  # £ per MW per year
wind_marginal_cost = 0.01  # £ per MWh (Variable O&M, assumed small)

n.add("Generator", "offshore_wind_farm",
      bus="electricity_bus",
      carrier="offshore_wind",
      p_nom_extendable=True,
      capital_cost=wind_capital_cost,
      marginal_cost=wind_marginal_cost,
      p_max_pu=wind_availability_pu)


# Generator (Gas Turbine Backup): The gas turbine provides backup, and its capacity is also extendable.
gas_capex_per_mw = 631000  # £ per MW
gas_lifetime_years = 25
gas_discount_rate = 0.07
annuitised_gas_cost = (gas_capex_per_mw * gas_discount_rate) / (1 - (1 + gas_discount_rate)**-gas_lifetime_years)
gas_fixed_opex = 12600  # £ per MW per year (fixed O&M)
gas_capital_cost = annuitised_gas_cost + gas_fixed_opex  # £ per MW per year

gas_fuel_price_mwh = 60  # £ per MWh_fuel (assumed fuel price)
gas_efficiency = 0.34  # Efficiency of gas turbine (MWh_elec / MWh_fuel)
gas_variable_operation_and_management_mwh = 7.05  # £ per MWh_elec (non-fuel variable O&M)
gas_marginal_cost_fuel = gas_fuel_price_mwh / gas_efficiency  # £ per MWh_elec
gas_marginal_cost = gas_marginal_cost_fuel + gas_variable_operation_and_management_mwh  # Total marginal cost

n.add("Generator", "gas_turbine",
      bus="electricity_bus",
      carrier="gas",
      p_nom_extendable=True,
      committable=False, # Cannot be both committable and extendable for LOPF capacity optimization
      capital_cost=gas_capital_cost,
      marginal_cost=gas_marginal_cost)


# --- DEFINE STORAGE (BESS) ---

# Storage (BESS - for sensitivity analysis): Battery storage parameters
bess_energy_capex_per_mwh = 225000  # £ per MWh
bess_power_capex_per_mw = 130000  # £ per MW
bess_lifetime_years = 15
bess_discount_rate = 0.07

annuitised_bess_energy_cost = (bess_energy_capex_per_mwh * bess_discount_rate) / (1 - (1 + bess_discount_rate)**-bess_lifetime_years)
bess_capital_cost_energy = annuitised_bess_energy_cost # Cost for energy capacity (Store)

annuitised_bess_power_cost = (bess_power_capex_per_mw * bess_discount_rate) / (1 - (1 + bess_discount_rate)**-bess_lifetime_years)
bess_fixed_opex_per_mw_power = 20000 # Fixed OPEX for power conversion part
bess_capital_cost_power = annuitised_bess_power_cost + bess_fixed_opex_per_mw_power # Cost for power capacity (Links)

bess_round_trip_efficiency = 0.87
standing_loss_per_hour = 0.001
bess_marginal_cost_links = 0.001 # Marginal cost for charging/discharging

# Store component representing the energy capacity of the BESS
n.add("Store", "bess_storage",
      bus="bess_connection_bus",
      carrier="battery", # This carrier should now be recognized
      e_nom_extendable=True,
      e_cyclic=True,
      standing_loss=standing_loss_per_hour,
      capital_cost=bess_capital_cost_energy)

# Link component for charging the BESS
n.add("Link", "bess_charger",
      bus0="electricity_bus",
      bus1="bess_connection_bus",
      carrier="battery", # This carrier should now be recognized
      p_nom_extendable=True,
      capital_cost=bess_capital_cost_power,
      efficiency=bess_round_trip_efficiency**0.5,
      marginal_cost=bess_marginal_cost_links)

# Link component for discharging the BESS
n.add("Link", "bess_discharger",
      bus0="bess_connection_bus",
      bus1="electricity_bus",
      carrier="battery", # This carrier should now be recognized
      p_nom_extendable=True,
      capital_cost=0, # Assuming power CAPEX is fully on charger link, or split if preferred
      efficiency=bess_round_trip_efficiency**0.5,
      marginal_cost=bess_marginal_cost_links)

# --- CONSISTENCY CHECK & SOLVE ---

# Perform consistency check
try:
    n.consistency_check()
    print("Network consistency check passed before solving.")
except UserWarning as e:
    print(f"Consistency check warning before solving: {e}") 
except Exception as e:
    print(f"Consistency check error before solving: {e}")
    # exit() # Optionally exit if errors are critical

# Ensure you have a solver installed (e.g., cbc, glpk, gurobi)
solver_name = 'cbc' # or 'glpk', 'gurobi', etc.
print(f"\nAttempting to solve the model with solver: {solver_name}")

try:
    # Solve the model using the optimize interface, common with linopy backend
    # The `solve_model` method handles the optimization.
    n.optimize(solver_name=solver_name)
    print("Model solved successfully.")

    # --- DISPLAY RESULTS ---
    print(f"\n--- Optimization Results ---")
    # Objective value is typically stored in n.model.objective after solving with n.optimize
    if hasattr(n, 'model') and hasattr(n.model, 'objective_value'):
         # Linopy stores objective in n.model.objective_value() or n.objective if populated by PyPSA post-solve
        obj_val = n.model.objective_value() if callable(n.model.objective_value) else n.model.objective_value
        # PyPSA also often populates n.objective directly
        if hasattr(n, 'objective'):
            obj_val = n.objective 
        print(f"Total System Cost (Objective Value): £{obj_val:,.2f} per year")

    else:
        print("Objective value not found. Solver may have failed or problem might be infeasible/unbounded.")

    print("\nOptimal Capacities:")
    # Check if components exist in results before trying to access them
    # Optimal capacities are in p_nom_opt (for Generators, Links) and e_nom_opt (for Stores)
    if 'offshore_wind_farm' in n.generators.index and 'p_nom_opt' in n.generators.columns:
        print(f"  Offshore Wind Farm (p_nom_opt): {n.generators.at['offshore_wind_farm', 'p_nom_opt']:.2f} MW")
    if 'gas_turbine' in n.generators.index and 'p_nom_opt' in n.generators.columns:
        print(f"  Gas Turbine (p_nom_opt): {n.generators.at['gas_turbine', 'p_nom_opt']:.2f} MW")
    if 'bess_storage' in n.stores.index and 'e_nom_opt' in n.stores.columns:
        print(f"  BESS Storage Energy Capacity (e_nom_opt): {n.stores.at['bess_storage', 'e_nom_opt']:.2f} MWh")
    if 'bess_charger' in n.links.index and 'p_nom_opt' in n.links.columns:
        print(f"  BESS Charger Power Capacity (p_nom_opt): {n.links.at['bess_charger', 'p_nom_opt']:.2f} MW")
    if 'bess_discharger' in n.links.index and 'p_nom_opt' in n.links.columns:
        print(f"  BESS Discharger Power Capacity (p_nom_opt): {n.links.at['bess_discharger', 'p_nom_opt']:.2f} MW")

except Exception as e:
    print(f"An error occurred during model solution or results processing: {e}")
    print("Please ensure you have a compatible solver installed and in your PATH.")
    print("Common solvers: cbc, glpk. You might need to install them.")
    print("Also ensure Linopy (PyPSA's modeling backend) is installed and functioning correctly.")



Network consistency check passed before solving.

Attempting to solve the model with solver: cbc


INFO:linopy.model: Solve problem using Cbc solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 7/7 [00:00<00:00, 242.05it/s]
INFO:linopy.io: Writing time: 0.22s
INFO:linopy.solvers:Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Aug 20 2024 

command line - cbc -printingOptions all -import /var/folders/r9/dprdmb3d7gs81f5ggtwv0lyw0000gn/T/linopy-problem-wlq6id_7.lp -solve -solu /var/folders/r9/dprdmb3d7gs81f5ggtwv0lyw0000gn/T/linopy-solve-5u9if3wr.sol (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 52560 (-61325) rows, 43804 (-8761) columns and 140160 (-78845) elements
Perturbing problem by 0.001% of 82.114679 - largest nonzero change 0.00018210041 ( 21.194816%) - largest zero change 0.00018143365
0  Obj 0 Primal inf 1106362.7 (8760)
487  Obj 64.630118 Primal inf 1101018.4 (8760)
974  Obj 129.29577 Primal inf 1096791.8 (8760)
1461  Obj 193.78578 Primal inf 1093583.9 (8760)
1948  Obj 258.20289 Primal i

Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
Model solved successfully.

--- Optimization Results ---
Objective value not found. Solver may have failed or problem might be infeasible/unbounded.

Optimal Capacities:
  Offshore Wind Farm (p_nom_opt): 220.34 MW
  Gas Turbine (p_nom_opt): 20.65 MW
  BESS Storage Energy Capacity (e_nom_opt): 196.71 MWh
  BESS Charger Power Capacity (p_nom_opt): 62.01 MW
  BESS Discharger Power Capacity (p_nom_opt): 70.94 MW


In [7]:
import plotly.express as px
# Get the wind power at each timestep in the problem
generator_power = n.generators_t.p
data_centre_power = n.loads_t.p["data_centre_load"]
#battery_power = n.links_t.p["bess_charger"] - n.links_t.p["bess_discharger"]
fig = px.line(
    data_frame=generator_power,
    x=generator_power.index,
    y=generator_power.columns,
    title="Generator Power Output Over Time",
    labels={"value": "Power (MW)", "variable": "Generator"},
    template="plotly_white"
)
fig.show()



In [8]:
total_gas_mwh_usage = n.generators_t.p["gas_turbine"].sum()
total_data_centre_mwh_usage = n.loads_t.p["data_centre_load"].sum()
print(f"Ratio of data centre load covered by gas turbine: {total_gas_mwh_usage / total_data_centre_mwh_usage:.2%}")

Ratio of data centre load covered by gas turbine: 1.78%


# Plan for the future
1. Initialise data centre load as a constant
2. Loop through wind farms with capacities at different multiple factors of the data centre.
3. Assume constant things like capex cost and capacity factor for wind. We can do a sensitivity analysis of these later.
4. v1: Assume no battery? v2: Assume battery as a MWh of the multiple of the data centre load too.
5. Solve the model to get the least cost way of operating it with gas + battery in each setup. Calculate the % of the total data centre load that was met by gas. Crude calcualtion = MWh of gas / MWh of data centre throughout the year?